<script src="//yihui.org/js/math-code.js"></script>
<!-- Just one possible MathJax CDN below. You may use others. -->
<script async
  src="//mathjax.rstudio.com/latest/MathJax.js?config=TeX-MML-AM_CHTML">
</script>
# AUTOGRAD: DIFERENCIAÇÃO AUTOMÁTICA

No centro de todas as redes neurais em PyTorch está o pacote `autograd`. Vamos primeiramente introduzir brevemente isto, e então partir para o treinamento da nossa primeira rede neural.

O pacote `autograd` fornece a diferenciação automática para todas operações em Tensores. É uma estrutura definida por execução (_define-by-run framework_), que significa que sua retropropagação é definida pela forma como seu código é executado e que toda iteração pode ser diferente.

Observemos de forma mais simples através de alguns exemplos.

## Tensor

`torch.Tensor` é a classe central do pacote. Se configurar seu atributo `.requires_grad` como `True` (verdadeiro), este começa a rastrear todas operações no mesmo. Quando finalizar sua computação, pode chamar `.backward()` e ter todos os gradientes computados automaticamente. O gradiente para este tensor será acumulado no atributo `.grad`.

Para parar o rastreamento do histórico de um tensor, podemos chamar `.detach()` para destacá-lo do histórico de computação, e evitar que a computação futura seja rastreada.

Para eviter rastreamento do histórico (e o uso da memória), podemos também empacotar o bloc de código em `with torch.no_grad():`. Isso pode ser particularmente de grande ajuda quando for avaliar um modelo, pois o modelo pode conter parâmetros treináveis com `requires_grad=True`, mas para o qual não precisamos dos gradientes.

Há mais uma classe muito importante para a implementação do autograd - uma `Function`.

`Tensor` e `Function` são interconectados e constroem um gráfico acíclico, que codifica um histórico completo da computação. Cada tensor tem um atributo `.grad_fn` que referencia uma `Function` que criara o `Tensor` (exceto para os Tensores criados pelo usuário - seus `grad_fn` são `None`).

Se quisermos computar os derivativos, podemos chamar `.backward()` em um `Tensor`. Se `Tensor` for um escalar (i.e., contém apenas um elemento), não precisamos especificar alrgumento algum ao `backward()`, no entanto, se tiver mais elementos, precisamos especificar um argumento `gradient` que é um tensor de dimensão correspondente.

In [20]:
import torch

Crie um tensor e configure `requires_grad=True` para rastrear a computação com isso.

In [21]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Faça uma operação de tensor.

In [22]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y` foi criado como um resultado de uma operação, então possui um `grad_fn`.

In [23]:
print(y.grad_fn)

Faça mais operações em `y`

In [24]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


`requires_grad_(...)` altera uma _flag_ `requires_grad` de um tensor existente de forma _in-place_. O _input_ (entrada) padrão da _flag_ é `False` se não fornecido.

In [25]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


## Gradientes

Vamos fazer a retropropagação agora. Como `out` contém apenas um valor escalar, `out.backward()` equivale a `out.backward(torch.tensor(1.))`.

Imprime gradientes _d(out)/dx_

In [26]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


O resultado foi uma matriz de `4.5`. Vamos chamar o _Tensor_ `out` de "$o$". Temos que $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ e $z_i\bigr\rvert_{x_i=1} = 27$.
Portanto,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, e consequentemente
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

Matematicamente, se tivermos uma função com valores vetoriais $\vec{y} = f(\vec{x})$, então o gradiente de $\vec{x}$ de $\vec{y}$ é uma matriz Jacobiana.

<!-- $$J = 
    \begin{pmatrix}
    \frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_1}{\partial x_n} \\
    \vdots & \ddots & \vdots \\
    \frac{\partial y_n}{\partial x_1} & \cdots & \frac{\partial y_n}{\partial x_n}$$ -->

Genericamente falando, `torch.autograd` é um motor para computar o produto do vetor Jacobiano. Isto é, dado um vetor $\vec{v}=(v_1 \space v_2 \cdots v_m)^T$, compute o produto $\vec{v}^T \bullet J$. Se $\vec{v}$ for gradiente de uma função escalar $l=g(\vec{y})$, i.e., $\vec{v} = \left(\frac{\partial l}{\partial y_1} \cdots \frac{\partial l}{\partial y_m}\right)$, então pela regra da cadeia, o produto do vetor Jacobiano seria o gradiente de $\vec{x}$ de $l$.

(Note que $\vec{v}^T \bullet J$ resulta em um vetor de uma linha, que pode ser tratado como um vetor de uma coluna invertendo a operação para $J^T \bullet \vec{v}$.)

Esta característica do produto do vetor Jacobiano é bastante conveniente para alimentar gradientes externos em um modelo que tem a _output_ (saída) não-escalar.

Para mais detalhes acesse [aqui](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html).

Agora, observemos um exemplo de produto do vetor Jacobiano:

In [27]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-723.9316,  522.3034, -503.7703], grad_fn=<MulBackward0>)


Neste caso, `y` não é mais um escalar. `torch.autograd` não pôde computar completamente o Jacobiano de forma direta, mas se quisermos apenas o produto do vetor Jacobiano, simplesmente passamos o vetor como argumento de `backward`:

In [28]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


Podemos também parar o autograd de rastrar o histórico nos Tensores com `.requires_grad=True`, seja empacotando o bloco de código em `with torch.no_grad():` ou usando `.detach()` para ter um novo Tensor com o mesmo conteúdo, mas que não requer os gradientes:

In [31]:
print("Empacotamento de bloco de código")
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)
    
print("\nNovo Tensor com mesmo conteúdo, mas sem histórico de gradientes.")
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

Empacotamento de bloco de código
True
True
False

Novo Tensor com mesmo conteúdo, mas sem histórico de gradientes.
True
False
tensor(True)


> Referência: [AUTOGRAD: AUTOMATIC DIFFERENTIATION](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py)